# Comparing Enactments

This notebook will show you how `legislice` provides convenient functions for comparing passages of legislative text.

As explained in the [downloading guide](https://legislice.readthedocs.io/en/latest/guides/downloading.html), begin by creating a `Client` to download and create `Enactment` objects.

In [1]:
import os
from dotenv import load_dotenv
from legislice.download import Client

load_dotenv()

TOKEN = os.getenv("LEGISLICE_API_TOKEN")
client = Client(api_token=TOKEN)

## Features of an Enactment

Now let's load the Fourteenth Amendment. You can get its effective date as a Python `date` object.

In [2]:
fourteenth_amendment = client.read(query="/us/const/amendment/XIV")
fourteenth_amendment.start_date

datetime.date(1868, 7, 28)

You can also view its `level`, which is "constitution", as opposed to "statute" or "regulation".

In [3]:
fourteenth_amendment.level

'constitution'

And you can find out the `node`, or place in the document tree, for the Fourteenth Amendment itself as well at its subsections.

In [4]:
fourteenth_amendment.node

'/us/const/amendment/XIV'

In [5]:
fourteenth_amendment.children[0].node

'/us/const/amendment/XIV/1'

You can also isolate parts of the `node` path, such as the `jurisdiction` and `code` the amendment comes from.

In [6]:
fourteenth_amendment.jurisdiction

'us'

In [7]:
fourteenth_amendment.code

'const'

## Selecting text

When you use the `selected_text` method, you get all the enacted text in the Fourteenth Amendment, including all its subsections.

In [8]:
fourteenth_amendment.selected_text()

'All persons born or naturalized in the United States, and subject to the jurisdiction thereof, are citizens of the United States and of the State wherein they reside. No State shall make or enforce any law which shall abridge the privileges or immunities of citizens of the United States; nor shall any State deprive any person of life, liberty, or property, without due process of law; nor deny to any person within its jurisdiction the equal protection of the laws. Representatives shall be apportioned among the several States according to their respective numbers, counting the whole number of persons in each State, excluding Indians not taxed. But when the right to vote at any election for the choice of electors for President and Vice President of the United States, Representatives in Congress, the Executive and Judicial officers of a State, or the members of the Legislature thereof, is denied to any of the male inhabitants of such State, being twenty-one years of age, and citizens of t

However, you might want an `Enactment` object that only represents a part of the Fourteenth Amendment that's relevant to a particular case. You can use the `select` method to limit the text of the provision that's considered "selected". One way to do this is with a series of strings that exactly match the text you want to select. 

In [9]:
fourteenth_amendment.select(["No State shall", "deprive any person of", "liberty", "without due process of law"])

Now that you've selected only some of the text, the output of the `selected_text` method will be different.

In [10]:
fourteenth_amendment.selected_text()

'…No State shall…deprive any person of…liberty…without due process of law…'

Every time you use the `select` method, it clears any existing text selection from the Enactment. But if you want to select additional text without clearing the existing selection, you can use `select_more`. It's okay if the selection you pass in to `select_more` overlaps with text you've already selected.

In [11]:
fourteenth_amendment.select_more("life, liberty, or property,")
fourteenth_amendment.selected_text()

'…No State shall…deprive any person of life, liberty, or property, without due process of law…'

If you need to select a passage that occurs more than once in the Enactment, you can import the `TextQuoteSelector` class instead of using strings. With a `TextQuoteSelector`, you specify not just the `exact` phrase you want to select, but also a `prefix` or `suffix` that makes the phrase uniquely identifiable. In this example, the text being selected is the second instance of the phrase "twenty-one years of age" in the Fourteenth Amendment.

In [12]:
from legislice.enactments import TextQuoteSelector
fourteenth_amendment.select(TextQuoteSelector(prefix="male citizens ", exact="twenty-one years of age"))

In [13]:
fourteenth_amendment.selected_text()

'…twenty-one years of age…'

You can also access the start and endpoints of the quoted passages, but there's a potential source of confusion: the `selection` attribute only provides you with the selected parts of the current node, not of the child nodes. For the Fourteenth Amendment, that will return an empty set because all of the Fourteenth Amendment's text is nested within numbered sections.

In [14]:
fourteenth_amendment.selection

TextPositionSet{}

To see the positions of the selected text in the child nodes as well, you need to use `tree_selection`. This is a method, not an attribute, so you need to include the parentheses at the end of the statement to make it work. In this example, the selected phrase "twenty-one years of age" starts on the 1254th character of the Fourteenth Amendment's text.

In [15]:
fourteenth_amendment.tree_selection()

TextPositionSet{TextPositionSelector[1254, 1277)}

You could also look at the `selection` attributes of the child nodes to see the positions of their selected text. But note: if you access the `selection` attribute on section 2 of the Fourteenth Amendment, then the starting index of the selected passage is counted from the beginning of section 2, not from the beginning of the entire Fourteenth Amendment.

In [16]:
fourteenth_amendment.children[1].selection

TextPositionSet{TextPositionSelector[786, 809)}

If you happen to know the start and end indices of the passage you want, then you can use a `TextPositionSelector` or `TextPositionSet` to select it, instead of specifying the exact text.

In [17]:
from legislice.enactments import TextPositionSelector, TextPositionSet

fourteenth_amendment.select(TextPositionSet([TextPositionSelector(1921, 1973), TextPositionSelector(2111, 2135)]))

In [18]:
fourteenth_amendment.selected_text()

'…The validity of the public debt of the United States…shall not be questioned.…'

## Comparing selected text

Legislice provides methods for comparing the selected text in Enactments. To get started, I'll use Python's `deepcopy` function to make a copy of the Enactment I was working on at the end of section 2. (If I used regular `copy` instead of `deepcopy`, then making changes to the copy could cause changes to the original, which would be confusing.)

In [19]:
from copy import deepcopy

public_debt_provision = deepcopy(fourteenth_amendment.children[3])
public_debt_provision.selected_text()

'The validity of the public debt of the United States…shall not be questioned.…'

Next, I'll change the selected text of the original `Enactment` to include all the text that was selected before, plus more.

In [20]:
fourteenth_amendment.select(TextPositionSelector(1921, 2135))
fourteenth_amendment.selected_text()

'…The validity of the public debt of the United States, authorized by law, including debts incurred for payment of pensions and bounties for services in suppressing insurrection or rebellion, shall not be questioned.…'

Now I can compare the text selections in these two Enactments. The `implies` method checks whether the Enactment on the left has all the text of the Enactment on the right. The `means` method checks whether they both have the same text.

In [21]:
fourteenth_amendment.implies(public_debt_provision)

True

You can also use Python's built-in "greater than or equal" operator as an alias for the `implies` method.

In [22]:
fourteenth_amendment >= public_debt_provision

True

Notice that Legislice is able to compare these two passages even though `amendment` is a text selection from the entire Fourteenth Amendment, while `public_debt_provision` is a text selection from only section 4 of the Fourteenth Amendment. You can verify this by checking the "node" attribute on each Enactment.

In [23]:
fourteenth_amendment.node

'/us/const/amendment/XIV'

In [24]:
public_debt_provision.node

'/us/const/amendment/XIV/4'

If you want to determine whether two Enactments have the same text (and neither has any more than the other), use the `means` method. Here's how you can check that the Fifth Amendment doesn't have identical text to the first section of the Fourteenth Amendment.

In [25]:
fifth_amendment = client.read(query="/us/const/amendment/V")
fifth_amendment.selected_text()

'No person shall be held to answer for a capital, or otherwise infamous crime, unless on a presentment or indictment of a Grand Jury, except in cases arising in the land or naval forces, or in the Militia, when in actual service in time of War or public danger; nor shall any person be subject for the same offence to be twice put in jeopardy of life or limb; nor shall be compelled in any Criminal Case to be a witness against himself; nor be deprived of life, liberty, or property, without due process of law; nor shall private property be taken for public use, without just compensation.'

In [26]:
fourteenth_amendment_section_1 = client.read(query="/us/const/amendment/XIV/1")
fifth_amendment.means(fourteenth_amendment_section_1)

False

However, the Fifth Amendment and the first section of the Fourteenth Amendment both happen to contain the phrase "life, liberty, or property, without due process of law". If you select that same passage from both provisions, then you can use the `means` method to verify that both text selections are identical.

In [27]:
phrase = "life, liberty, or property, without due process of law"
fourteenth_amendment_section_1.select(phrase)
fifth_amendment.select(phrase)
fourteenth_amendment_section_1.means(fifth_amendment)

True

There are many situations in real legal analysis where it's helpful to know if identical text has been enacted at different citations. It could mean that the identical section has been renumbered, or it could mean that a judicial interpretation of one Enactment is also relevant to the other Enactment. Legislice's `implies` and `means` methods can help you automate that analysis.

Since you can use `>=` as an alias for `implies`, you might expect to be able to use `==` as an alias for `means`. Currently you can't do that, because overriding the equals function could interfere with Python's ability to determine what objects are identical, and could cause bugs that would be difficult to diagnose. However, you can use `>` as an alias that returns `True` only if `implies` would return `True` while `means` would return `False`.

## Combining Enactments

When you have two Enactments and either they are at the same node or one is a descendant of the other, you can combine them into a new Enactment using the addition sign. Here's an example from a copyright statute in the United States Code. The example shows that you can load section `/us/usc/t17/s103`, select some text from subsection `b` of that provision, and then add it to a separate Enactment representing the entirety of subsection `/us/usc/t17/s103/a`. Legislice combines the text from subsection `a` and subsection `b` in the correct order.

In [28]:
s103 = client.read(query="/us/usc/t17/s103", date="2020-01-01")
selections = ["The copyright in such work is independent of",
              "any copyright protection in the preexisting material."]
s103.select(selections)

In [29]:
s103.selected_text()

'…The copyright in such work is independent of…any copyright protection in the preexisting material.'

In [30]:
s103a = client.read(query="/us/usc/t17/s103/a", date="2020-01-01")
s103a.selected_text()

'The subject matter of copyright as specified by section 102 includes compilations and derivative works, but protection for a work employing preexisting material in which copyright subsists does not extend to any part of the work in which such material has been used unlawfully.'

In [31]:
combined_enactment = s103 + s103a
combined_enactment.selected_text()

'The subject matter of copyright as specified by section 102 includes compilations and derivative works, but protection for a work employing preexisting material in which copyright subsists does not extend to any part of the work in which such material has been used unlawfully.…The copyright in such work is independent of…any copyright protection in the preexisting material.'

## EnactmentGroups

When you want to work with groups of Enactments that may or may not be nested inside one another, you can put them together in an EnactmentGroup. When you create an EnactmentGroup or add two EnactmentGroups together, any overlapping Enactments inside will be combined into a single Enactment.

In this example, we create two EnactmentGroups called `left` and `right`, each containing two Enactments. Because one of the Enactments in `left` overlaps with one of the Enactments in `right`, when we add `left` and `right` together those two Enactments will be combined into one. Thus the resulting EnactmentGroup will contain three Enactments instead of four.

In [32]:
from legislice import EnactmentGroup

establishment_clause = client.read(query="/us/const/amendment/I")
establishment_clause.select("Congress shall make no law respecting an establishment of religion")
speech_clause = client.read(query="/us/const/amendment/I")
speech_clause.select(["Congress shall make no law", "abridging the freedom of speech"])

arms_clause = client.read(query="/us/const/amendment/II")
arms_clause.select("the right of the people to keep and bear arms, shall not be infringed.")
third_amendment = client.read(query="/us/const/amendment/III")

left = EnactmentGroup([establishment_clause, arms_clause])
right = EnactmentGroup([third_amendment, speech_clause])

combined = left + right

In [33]:
print(combined)

the group of Enactments:
  "Congress shall make no law respecting an establishment of religion…abridging the freedom of speech…" (/us/const/amendment/I 1791-12-15)
  "…the right of the people to keep and bear arms, shall not be infringed." (/us/const/amendment/II 1791-12-15)
  "No soldier shall, in time of peace be quartered in any house, without the consent of the Owner, nor in time of war, but in a manner to be prescribed by law." (/us/const/amendment/III 1791-12-15)


In [34]:
len(combined)

3

## Converting Enactments to JSON

When you want a representation of a legislative passage that's precise, machine-readable, and easy to share over the internet, you can use Legislice's JSON schema. Here's how to convert the Enactment object called `combined_enactment`, which was created in the example above, to JSON. 

This JSON represents a selection of three nonconsecutive passages from the most recent version of section 103 of Title 17 of the United States Code. The `.dumps()` method returns a JSON string, while the `.dump()` method returns a Python dictionary.

In [35]:
from legislice.schemas import EnactmentSchema

schema = EnactmentSchema()
schema.dumps(combined_enactment)

'{"node": "/us/usc/t17/s103", "heading": "Subject matter of copyright: Compilations and derivative works", "text_version": null, "start_date": "2013-07-18", "end_date": null, "known_revision_date": false, "selection": [], "anchors": [], "children": [{"node": "/us/usc/t17/s103/a", "heading": "", "text_version": {"content": "The subject matter of copyright as specified by section 102 includes compilations and derivative works, but protection for a work employing preexisting material in which copyright subsists does not extend to any part of the work in which such material has been used unlawfully."}, "start_date": "2013-07-18", "end_date": null, "known_revision_date": false, "selection": [{"start": 0, "end": 277}], "anchors": [], "children": []}, {"node": "/us/usc/t17/s103/b", "heading": "", "text_version": {"content": "The copyright in a compilation or derivative work extends only to the material contributed by the author of such work, as distinguished from the preexisting material empl

## Formatting Citations (Experimental)

Legislice has preliminary support for serializing citations for Enactment objects based on [Citation Style Language JSON](https://citeproc-js.readthedocs.io/en/latest/csl-json/markup.html). The goal of this feature is to support compatibility with [Jurism](https://juris-m.github.io/). Please [open an issue in the Legislice repo](https://github.com/mscarey/legislice/issues) if you have suggestions for how this feature should develop to support your use case.

This CSL-JSON format currently only identifies the cited provision down to the section level. A citation to a subsection or deeper nested provision will be the same as a citation to its parent section.

In [36]:
cares_act_benefits = client.read("/us/usc/t15/s9021/")
cares_act_benefits.heading

'Pandemic unemployment assistance'

In [37]:
citation = cares_act_benefits.as_citation()
str(citation)

'15 U.S. Code § 9021 (2020)'

In [38]:
cares_act_benefits.csl_json()

'{"type": "legislation", "jurisdiction": "us", "container-title": "U.S. Code", "volume": "15", "section": "sec. 9021", "event-date": {"date-parts": [["2020", 4, 10]]}}'